[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fnhirwa/gsoc2024/blob/main/examples/darts_examples.ipynb)

# Darts Regression models for time series forecasting using sktime

This notebook demonstrates how you can simply use the `sktime` to interface the `darts` regression models for time series forecasting. We will explore with the current adaptation of the `darts` models to the `sktime` interface and how to use them for forecasting.

### Installing the required libraries


In [1]:
%%capture
# sktime installation with darts dependencies
! pip install -U sktime
! pip install -U u8darts

In [2]:
# importing needed libraries
import copy

from sklearn.linear_model import LinearRegression
from sktime.datasets import load_longley
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.darts import (
    DartsLinearRegressionModel,
    DartsRegressionModel,
    DartsXGBModel,
)
from darts.datasets import WeatherDataset
from sktime.split import temporal_train_test_split

### Loading the data

In [3]:
y, X = load_longley()
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X, test_size=4)

### Fitting the model

#### Defining the models

In [4]:
# DartsLinearRegressionModel
dartslr = DartsLinearRegressionModel(
    lags=6,
    output_chunk_length=4,
    kwargs={
        "fit_intercept": True,
    },
)
fh = ForecastingHorizon([1, 2, 3, 4], is_relative=True)

# DartsRegressionModel
dartsr = DartsRegressionModel(
    lags=6,
    output_chunk_length=4,
    model=LinearRegression(),
)

# DartsXGBModel
dartsxgb = DartsXGBModel(
    lags=6,
    output_chunk_length=4,
    kwargs={
        "n_estimators": 100,
        "max_depth": 3,
        "objective": "reg:squarederror",
        "eval_metric": "mae",
    },
)

#### Fitting models to load_longley dataset

In [5]:
dartslr.fit(y_train, fh=fh)
dartsr.fit(y_train, fh=fh)
dartsxgb.fit(y_train, fh=fh)


Support for Torch based models not available. To enable them, install "darts", "u8darts[torch]" or "u8darts[all]" (with pip); or "u8darts-torch" or "u8darts-all" (with conda).


DartsXGBModel(kwargs={'eval_metric': 'mae', 'max_depth': 3, 'n_estimators': 100,
                      'objective': 'reg:squarederror'},
              lags=6, output_chunk_length=4)

#### Fitting on darts.datasets Weather dataset


In [6]:
series = WeatherDataset().load()
target = series["p (mbar)"][:100]
target_df = target.pd_series()

darts_weather_lr = copy.deepcopy(dartslr)
darts_weather_r = copy.deepcopy(dartsr)
darts_weather_xgb = copy.deepcopy(dartsxgb)

In [7]:
darts_weather_lr.fit(target_df)
darts_weather_r.fit(target_df)
darts_weather_xgb.fit(target_df)

DartsXGBModel(kwargs={'eval_metric': 'mae', 'max_depth': 3, 'n_estimators': 100,
                      'objective': 'reg:squarederror'},
              lags=6, output_chunk_length=4)

### Making predictions

#### Making predictions on load_longley dataset

In [8]:
pred_fh = ForecastingHorizon([1, 2], is_relative=True)
preds_dartslr = dartslr.predict(fh=pred_fh)
preds_dartsr = dartsr.predict(fh=pred_fh)
preds_dartsxgb = dartsxgb.predict(fh=pred_fh)

In [9]:
print("DartsLinearRegressionModel predictions\n")
print(preds_dartslr)
print("\nDartsRegressionModel predictions\n")
print(preds_dartsr)
print("\nDartsXGBModel predictions\n")
print(preds_dartsxgb)

DartsLinearRegressionModel predictions

1959    65651.834793
1960    75258.144718
Freq: Y-DEC, Name: TOTEMP, dtype: float64

DartsRegressionModel predictions

1959    65651.834793
1960    75258.144718
Freq: Y-DEC, Name: TOTEMP, dtype: float64

DartsXGBModel predictions

1959    63761.011719
1960    66019.007812
Freq: Y-DEC, Name: TOTEMP, dtype: float32


#### Making predictions on darts.datasets Weather dataset

In [10]:
pred_fh = list(range(1, 5))
preds_weather_darts_lr = darts_weather_lr.predict(fh=pred_fh)
preds_weather_darts_r = darts_weather_r.predict(fh=pred_fh)
preds_weather_darts_xgb = darts_weather_xgb.predict(fh=pred_fh)

In [11]:
print("DartsLinearRegressionModel predictions\n")
print(preds_weather_darts_lr)
print("\nDartsRegressionModel predictions\n")
print(preds_weather_darts_r)
print("\nDartsXGBModel predictions\n")
print(preds_weather_darts_xgb)

DartsLinearRegressionModel predictions

2020-01-01 16:50:00    1005.729880
2020-01-01 17:00:00    1005.688367
2020-01-01 17:10:00    1005.649921
2020-01-01 17:20:00    1005.640799
Freq: 10min, Name: p (mbar), dtype: float64

DartsRegressionModel predictions

2020-01-01 16:50:00    1005.729880
2020-01-01 17:00:00    1005.688367
2020-01-01 17:10:00    1005.649921
2020-01-01 17:20:00    1005.640799
Freq: 10min, Name: p (mbar), dtype: float64

DartsXGBModel predictions

2020-01-01 16:50:00    1005.803833
2020-01-01 17:00:00    1005.724243
2020-01-01 17:10:00    1005.716980
2020-01-01 17:20:00    1005.714783
Freq: 10min, Name: p (mbar), dtype: float32


### Conclusion

In this notebook, we demonstrate how you can simply perform time series forecasting using the `darts` models through the `sktime` interface. We have shown how to fit the models on the `load_longley` and `darts.datasets` Weather dataset and make predictions. For more details on sktime interface to darts, please refer to the [sktime darts adapters](https://www.sktime.net/en/stable/api_reference/forecasting.html#direct-and-recursive-3rd-party) documentation.

### References

- [sktime](https://www.sktime.net/en/stable/index.html)
- [darts](https://unit8co.github.io/darts/index.html)